<div class="alert alert-block alert-info" style="color:black"> <h2>Activity 5: Investigate the time and space (memory) requirements of your two methods</h2>
    You should now have working versions of both breadth-first and (restricted) depth-first search. They already store the number of attempts tested (a measure of runtime), and the code cells that run them print that value out.<br>
    The next step is to compare memory - which is proportional to the maximum size of the open list. 
    <br><br><b>How to get started:</b> Edit your code for both classes:
    <ol>
    <li> Copy-paste <code>update_working_memory()</code> into your <code>BreadthFirstSearch</code> class</li>
    <li> In both your classes add a new parameter <code>self.max_memory</code> with a default value 0 by over-riding the <code>__init__()</code> method of the super class.</ul>
    <li> Override <code>update_working_memory()</code> in both your classes, adding code to:
        <ul>
            <li>check the length of the open_list against <code>self.max_memory</code></li>
            <li> update the value of <code>self.max_memory</code> if the open list has increased in size.</li>
        </ul>
    <li> Copy-paste the testing code from the cells above, then adapt it to test the time and memory needs of your algorithms.</li> 
    </ol>
    <b>Note:</b> this is a <em>Stretch</em> activity so don't worry if you can't complete it easily.
</div>

In [ ]:
# ====> insert your code here
import sys, os

# Import from the common directory
sys.path.append('../common')
from problem import Problem
from candidatesolution import CandidateSolution
from singlemembersearch import SingleMemberSearch
from combinationproblem import CombinationProblem
from foxchickengrain import FoxChickenGrain

class BreadthFirstSearch(SingleMemberSearch):
    """Implementation of breadth first search to extend
    the superclass SingleMemberSearch search.
    Adds a __str__ method
    Over-rides the method select_and_move_from_openlist
    to implement the algorithm, and tracks memory usage
    """

    def __init__(self, problem, constructive=False, max_attempts=1000):
        """Initialize with additional memory tracking"""
        super().__init__(problem, constructive, max_attempts)
        self.max_memory = 0  # Track maximum open list size

    def __str__(self):
        return "breadth-first"

    def select_and_move_from_openlist(self) -> CandidateSolution:
        """Implements the breadth-first search algorithm

        Returns
        -------
        next working candidate (solution) taken from open list
        """
        # create a candidate solution variable to hold the next solution
        next_soln = CandidateSolution()

        # my_index ← GetFirstIndex(open_list)
        my_index = 0

        # the_candidate ← open_list(my_index)
        next_soln = self.open_list[my_index]

        # RemoveFromOpenList(my_index)
        self.open_list.pop(my_index)

        return next_soln

    def update_working_memory(self, neighbour, status):
        """decides what to do with a neighbour based on its status
        over-ridden to track memory usage

        Parameters
        ----------
        neighbour : CandidateSolution
            the solution being classified
        status : int
            result of evaluating the neighbour

        Returns
        -------
        Boolean
            True if we found the goal and need to break out of the loop
            False otherwise
        """
        if status == 1:  # This is the goal found status (1)
            self.result = neighbour
            self.solved = True
            return True
        elif status == -1:  # This is the infeasible status (-1)
            self.closed_list.append(neighbour)
        else:
            self.open_list.append(neighbour)
            # Track maximum memory usage (open list size)
            current_memory = len(self.open_list)
            if current_memory > self.max_memory:
                self.max_memory = current_memory
        return False


class RestrictedDepthFirstSearch(SingleMemberSearch):
    """Implementation of depth first search with a maximum depth limit
    Extends the SingleMemberSearch class.
    Adds a __str__ method
    Over-rides the methods:
    - __init__ to add max_depth parameter and memory tracking
    - select_and_move_from_openlist to implement depth-first selection
    - update_working_memory to restrict depth of search and track memory
    """

    def __init__(self, problem, constructive=False, max_attempts=1000, max_depth=10):
        """Initialize with an additional max_depth parameter and memory tracking"""
        super().__init__(problem, constructive, max_attempts)
        self.max_depth = max_depth
        self.max_memory = 0  # Track maximum open list size

    def __str__(self):
        return "restricted-depth-first"

    def select_and_move_from_openlist(self) -> CandidateSolution:
        """Implements the depth-first search algorithm

        Returns
        -------
        next working candidate (solution) taken from open list
        """
        # create a candidate solution variable to hold the next solution
        next_soln = CandidateSolution()

        # my_index ← GetLastIndex(open_list)
        my_index = len(self.open_list) - 1

        # the_candidate ← open_list(my_index)
        next_soln = self.open_list[my_index]

        # RemoveFromOpenList(my_index)
        self.open_list.pop(my_index)

        return next_soln

    def update_working_memory(self, neighbour, status):
        """decides what to do with a neighbour based on its status
        over-ridden to implement depth restriction and track memory

        Parameters
        ----------
        neighbour : CandidateSolution
            the solution being classified
        status : int
            result of evaluating the neighbour

        Returns
        -------
        Boolean
            True if we found the goal and need to break out of the loop
            False otherwise
        """
        if status == 1:  # This is the goal found status (1)
            self.result = neighbour
            self.solved = True
            return True
        elif status == -1:  # This is the infeasible status (-1)
            self.closed_list.append(neighbour)
        elif len(neighbour.variable_values) < self.max_depth:
            # Only add to open list if depth is less than max_depth
            self.open_list.append(neighbour)
            # Track maximum memory usage (open list size)
            current_memory = len(self.open_list)
            if current_memory > self.max_memory:
                self.max_memory = current_memory
        return False

In [10]:
# Create a FoxChickenGrain problem
myproblem = FoxChickenGrain()

# Define parameters for testing
max_attempts = 1000
depth_values = [7, 8, 9, 10]  # Different depth limits to test

print("Comparing search algorithms on the Fox-Chicken-Grain problem:")
print("=" * 70)
print(f"{'Algorithm':<25} {'Attempts':<10} {'Max Memory':<12} {'Solution Length':<15} {'Found?':<8}")
print("-" * 70)

# Test breadth-first search
my_breadth_search = BreadthFirstSearch(myproblem, constructive=True, max_attempts=max_attempts)
found = my_breadth_search.run_search()
solution_length = len(my_breadth_search.result.variable_values) if found else "N/A"
print(f"{my_breadth_search.__str__():<25} {my_breadth_search.trials:<10} {my_breadth_search.max_memory:<12} {solution_length:<15} {found}")

# Test restricted depth-first search with different depth limits
for depth in depth_values:
    my_depth_search = RestrictedDepthFirstSearch(
        myproblem,
        constructive=True,
        max_attempts=max_attempts,
        max_depth=depth
    )
    found = my_depth_search.run_search()
    solution_length = len(my_depth_search.result.variable_values) if found else "N/A"
    algorithm_name = f"{my_depth_search.__str__()}(depth={depth})"
    print(f"{algorithm_name:<25} {my_depth_search.trials:<10} {my_depth_search.max_memory:<12} {solution_length:<15} {found}")

print("\nDetailed solutions for successful searches:")
print("=" * 70)

# Print solutions for successful searches
my_breadth_search = BreadthFirstSearch(myproblem, constructive=True, max_attempts=max_attempts)
found = my_breadth_search.run_search()
if found:
    print(f"\nBreadth-first search:")
    print(f"  Solved after {my_breadth_search.trials} attempts")
    print(f"  Maximum open list size: {my_breadth_search.max_memory}")
    print(f"  Solution length: {len(my_breadth_search.result.variable_values)}")
    print(f"  Solution: {myproblem.display(my_breadth_search.result)}")

# Test optimal depth for restricted depth-first search
for depth in range(7, 11):
    my_depth_search = RestrictedDepthFirstSearch(
        myproblem,
        constructive=True,
        max_attempts=max_attempts,
        max_depth=depth
    )
    found = my_depth_search.run_search()
    if found:
        print(f"\nRestricted depth-first search (max_depth={depth}):")
        print(f"  Solved after {my_depth_search.trials} attempts")
        print(f"  Maximum open list size: {my_depth_search.max_memory}")
        print(f"  Solution length: {len(my_depth_search.result.variable_values)}")
        print(f"  Solution: {myproblem.display(my_depth_search.result)}")
        break  # Found the minimum depth that works

Comparing search algorithms on the Fox-Chicken-Grain problem:
Algorithm                 Attempts   Max Memory   Solution Length Found?  
----------------------------------------------------------------------
breadth-first             1001       876          N/A             False
restricted-depth-first(depth=7) 1001       43           N/A             False
restricted-depth-first(depth=8) 1001       50           N/A             False
restricted-depth-first(depth=9) 1001       57           N/A             False
restricted-depth-first(depth=10) 1001       64           N/A             False

Detailed solutions for successful searches:


In [ ]:
import workbook2_utils as wb2
display(wb2.q12)
display(wb2.q13)